In [166]:
import pandas as pd
import numpy as np

In [263]:
colunas = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv("car.data", names=colunas)

In [126]:
def entropia(c):
  # normaliza o vetor
  c_normalized = c / float(np.sum(c))
  # seleciona apenas valores diferentes de 0
  c_normalized = c_normalized[np.nonzero(c_normalized)]
  # calculo da entropia
  H = -sum(c_normalized*np.log2(c_normalized))  
  return H

def selecao_treino_teste(dataset, prct =0.5):
  """
  Divide a base em treino e teste

  Params:
    - dataset: dataframe do pandas
    - prct: porcentagem da base de treino a ser selecionada
  return:
    - df_treino
    - df_teste
  """
  indices = dataset.index
  size = int(prct*len(dataset))
  # escolhe os indices de treino aleatoriamente
  idx_treino = np.random.choice(indices,size=size, replace=False)
  # obtem os indices de teste
  idx_teste = list(set(indices).difference(idx_treino))
  
  # selecao dos dataframes de treino e teste
  df_treino = dataset.iloc[idx_treino]
  df_teste = dataset.iloc[idx_teste]
  
  return df_treino, df_teste
  
def acuracia(y_real, y_pred):
    comparacoes = y_real==y_pred
    acc = (sum(comparacoes)/len(comparacoes))*100
    return acc

In [1]:
def ganho(df, v, ht, tglabel=None):
    """
    Calcula o ganho de informação
    """
    counts_c = df[v].value_counts()
    classes = list(counts_c.index)
    gain = ht
    for c in classes:
        vc = df[df[v]==c]
        prop_v = np.array(vc[tglabel].value_counts()/vc.shape[0])
        htv = entropia(prop_v)
        shtv = (counts_c.loc[c]/df.shape[0])*htv    
        gain -= shtv
    return gain

def get_root(df, variaveis, tglabel=None):
    """
    Verifica qual atributo será a próxima raiz
    """
    ht = entropia(np.array(df[tglabel].value_counts()/df.shape[0]))    
    ganhos = [ganho(df, v, ht, tglabel) for v in variaveis]
    return variaveis[np.argmax(ganhos)]

def build_tree(df, variaveis, tg_label=6, treedepth=0):
    """
    Constroí a arvore de decisão usando o ganho de informação
    """

    root = {}    
    prop = df[tg_label].value_counts()/df.shape[0]
    # retorna um no folha
    if (len(variaveis)==0) or (treedepth>=3):
        classes, counts = np.unique(df[tg_label], return_counts=True)
        root[classes[np.argmax(counts)]] = {}
        return root
    # retorna um no folha com uma única decisão
    elif(len(prop)==1):
        root[prop.index[0]] = {}
        return root
    else:
        r = get_root(df, variaveis, tg_label)
        V = df[r].unique()
        root.setdefault(r,{})
        for vi in V:
            # cria uma nova ramificação
            root[r][vi] = {}
            df_vi = df[df[r]==vi]
            if (len(df_vi)==0):
                classes, counts = np.unique(df[tg_label], return_counts=True)
                classe = classes[np.argmax(counts)]
                root[r][vi] = classe
            else:
                variaveis = list(set(variaveis).difference([r]))
                root[r][vi] = build_tree(df_vi, variaveis, tg_label=tg_label, treedepth=treedepth+1)
    return root

def predict(tree, root, x):
    """
    Realiza a predição usando uma árvore
    """
    keys = list(tree[root].keys())
    # se a raiz nao tem filhos é um no folha
    if len(keys)==0:
        return root
    # percorre a arvore recursivamente com base nos atributos da amostra
    tree = tree[root][x[root]]
    nr = list(tree.keys())[0]
    return predict(tree, nr, x) 

def print_tree(tree, root, vars={}, deepth=0):
    """
    Imprime os atributos de uma árvore (ignora as ramificações)
    """
    keys = list(tree[root].keys())
    vars.setdefault(deepth,[])
    vars[deepth].append(root)
    if(deepth>=2):
        return vars

    for k in keys:
        next_node = list(tree[root][k].keys())[0]
        ntree = tree[root][k]
        print_tree(ntree, next_node, vars, deepth=deepth+1)
    return vars

In [264]:
# separacao em teste e treino
df_treino, df_teste = selecao_treino_teste(car_data, 0.75)
tree_car = build_tree(df_treino, colunas[:-1], tg_label='class')

In [295]:
y_pred = []
for i in df_teste.index:
    y_pred.append(predict(tree_car, 'safety', df_teste.loc[i]))
acc = acuracia(y_pred, df_teste['class'].values)
print ("Acuracia", acc)

Acuracia 80.32407407407408


## Estrutura da arvore

In [281]:
tree_car

{'safety': {'med': {'persons': {'2': {'unacc': {}},
    'more': {'buying': {'vhigh': {'unacc': {}},
      'med': {'acc': {}},
      'low': {'acc': {}},
      'high': {'unacc': {}}}},
    '4': {'buying': {'low': {'acc': {}},
      'vhigh': {'unacc': {}},
      'high': {'unacc': {}},
      'med': {'acc': {}}}}}},
  'low': {'unacc': {}},
  'high': {'persons': {'2': {'unacc': {}},
    '4': {'buying': {'low': {'acc': {}},
      'high': {'acc': {}},
      'vhigh': {'acc': {}},
      'med': {'acc': {}}}},
    'more': {'buying': {'high': {'acc': {}},
      'med': {'acc': {}},
      'low': {'vgood': {}},
      'vhigh': {'unacc': {}}}}}}}}

# b)

# c)

In [296]:
# obtencao de 2 arvores
arvores = []
for i in range(2):
    df_treino, df_teste = selecao_treino_teste(car_data, 0.75)
    t = build_tree(df_treino, colunas[:-1], tg_label='class')
    arvores.append(t)

In [307]:
vars = {}
print_tree(tree_car, 'safety', vars)

{0: ['safety'],
 1: ['persons', 'unacc', 'persons'],
 2: ['unacc', 'buying', 'buying', 'unacc', 'buying', 'buying']}

In [308]:
print_tree(arvores[0], 'safety', vars={})

{0: ['safety'],
 1: ['persons', 'persons', 'unacc'],
 2: ['buying', 'buying', 'unacc', 'buying', 'buying', 'unacc']}

In [309]:
print_tree(arvores[1], 'safety', vars={})

{0: ['safety'],
 1: ['persons', 'persons', 'unacc'],
 2: ['buying', 'buying', 'unacc', 'buying', 'unacc', 'buying']}

### R:
A estrutura das árvores foram consistentes para os 2 níveis